In [1]:
# Load Libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import warnings
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [2]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Hyper parameters
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

# MNIST dataset

In [ ]:
current_path = os.getcwd()
print(current_path)

In [3]:
from six.moves import urllib
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)

In [8]:
train_dataset = datasets.MNIST(root=f'./data/',
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=False)

In [9]:
test_dataset = datasets.MNIST(root='data/',
                                          train=False,
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [10]:

# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

In [11]:

model = ConvNet(num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Epoch [1/5], Step [100/600], Loss: 0.1736
Epoch [1/5], Step [200/600], Loss: 0.0656
Epoch [1/5], Step [300/600], Loss: 0.0621
Epoch [1/5], Step [400/600], Loss: 0.0829
Epoch [1/5], Step [500/600], Loss: 0.1254
Epoch [1/5], Step [600/600], Loss: 0.0523
Epoch [2/5], Step [100/600], Loss: 0.0509
Epoch [2/5], Step [200/600], Loss: 0.0438
Epoch [2/5], Step [300/600], Loss: 0.0438
Epoch [2/5], Step [400/600], Loss: 0.0747
Epoch [2/5], Step [500/600], Loss: 0.0393
Epoch [2/5], Step [600/600], Loss: 0.0280
Epoch [3/5], Step [100/600], Loss: 0.0323
Epoch [3/5], Step [200/600], Loss: 0.0241
Epoch [3/5], Step [300/600], Loss: 0.0090
Epoch [3/5], Step [400/600], Loss: 0.0447
Epoch [3/5], Step [500/600], Loss: 0.0138
Epoch [3/5], Step [600/600], Loss: 0.0184
Epoch [4/5], Step [100/600], Loss: 0.0124
Epoch [4/5], Step [200/600], Loss: 0.0024
Epoch [4/5], Step [300/600], Loss: 0.0103
Epoch [4/5], Step [400/600], Loss: 0.0443
Epoch [4/5], Step [500/600], Loss: 0.0191
Epoch [4/5], Step [600/600], Loss: